### Importing Libraries

In [1]:
import numpy as np

import pandas as pd

# import json # library to handle JSON files
# from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests
import urllib.request

from bs4 import BeautifulSoup
from urllib.request import urlopen

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!pip install folium
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

### Data scraping

Now we scrape the Dataset of Torronto's neighbourhoods from a wikipedia link.

Canada has a total of 10 boroughs and 103 neighborhoods.
We segment the neighborhoods and explore them, we will essentially need a dataset that contains the 10 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

In [2]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=942851379'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')

page = urllib.request.urlopen(url)

soup = BeautifulSoup(page, "lxml")

soup.title.string

'List of postal codes of Canada: M - Wikipedia'

In [3]:
all_tab=soup.find_all('table')
all_tab

[<table class="wikitable sortable">
 <tbody><tr>
 <th>Postcode</th>
 <th>Borough</th>
 <th>Neighbourhood
 </th></tr>
 <tr>
 <td>M1A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M2A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M3A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td></tr>
 <tr>
 <td>M4A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td></tr>
 <tr>
 <td>M5A</td>
 <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
 <td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
 </td></tr>
 <tr>
 <td>M6A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
 </td></tr>
 <tr>


In [4]:
right_table=soup.find('table',class_='wikitable sortable')
right_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North

In [5]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [6]:
toronto_df=pd.DataFrame(A,columns=['PostCode'])
toronto_df['Borough']=B
toronto_df['Neighborhood']=C

#### We get the scraped data as follows :

In [7]:
toronto_df.head()

,PostCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Cleaning the data set

In [8]:
toronto_df=toronto_df[toronto_df["Borough"]!='Not assigned']

In [9]:
toronto_df.head()

,PostCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [10]:
toronto_df.reset_index(inplace=True)

In [11]:
toronto_df.drop('index',axis=1,inplace=True)

In [12]:
toronto_df.head()

,PostCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [13]:
toronto_df['Neighbourhood'] = toronto_df.groupby(['PostCode','Borough'])['Neighborhood'].transform(lambda x: ','.join(x))
toronto_df.drop_duplicates(inplace=True)
toronto_df.head()

,PostCode,Borough,Neighborhood,Neighbourhood
0,M3A,North York,Parkwoods,Parkwoods
1,M4A,North York,Victoria Village,Victoria Village
2,M5A,Downtown Toronto,Harbourfront,Harbourfront
3,M6A,North York,Lawrence Heights,"Lawrence Heights,Lawrence Manor"
4,M6A,North York,Lawrence Manor,"Lawrence Heights,Lawrence Manor"


In [14]:
toronto_df[toronto_df.Neighborhood=='Not assigned'].count()

PostCode         0
Borough          0
Neighborhood     0
Neighbourhood    0
dtype: int64

In [15]:
toronto_df.rename(columns={'PostCode':'Postal Code'},inplace=True)

In [16]:
toronto_df.head()

,Postal Code,Borough,Neighborhood,Neighbourhood
0,M3A,North York,Parkwoods,Parkwoods
1,M4A,North York,Victoria Village,Victoria Village
2,M5A,Downtown Toronto,Harbourfront,Harbourfront
3,M6A,North York,Lawrence Heights,"Lawrence Heights,Lawrence Manor"
4,M6A,North York,Lawrence Manor,"Lawrence Heights,Lawrence Manor"


In [17]:
toronto_df['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

#### Adding the longitude and Latitude values

In [18]:
longlat=pd.read_csv("http://cocl.us/Geospatial_data")
longlat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
toronto_df=pd.merge(toronto_df,longlat,on='Postal Code')

In [20]:
toronto_df.count()

Postal Code      210
Borough          210
Neighborhood     210
Neighbourhood    210
Latitude         210
Longitude        210
dtype: int64

#### Making sure that the dataset has all 10 boroughs and 103 neighborhoods.

In [21]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df['Borough'].unique()),
        toronto_df.shape[0]
    )
)

The dataframe has 10 boroughs and 210 neighborhoods.


#### Using geopy library to get the latitude and longitude values of Toronto.


In [22]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Creating a map of Toronto with neighborhoods superimposed on top.

In [23]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Defining Foursquare Credentials and Version

In [24]:
CLIENT_ID = '1EH01SK1KF05C4GKBFLMWEPRE2EWBS10TKXQE4X3DQUELTFA' # your Foursquare ID
CLIENT_SECRET = 'VJFZ4UL3PP0ZHZS3G4IFIVTBSSUSM5QIFLTMLMPJYAJEMRWG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1EH01SK1KF05C4GKBFLMWEPRE2EWBS10TKXQE4X3DQUELTFA
CLIENT_SECRET:VJFZ4UL3PP0ZHZS3G4IFIVTBSSUSM5QIFLTMLMPJYAJEMRWG


<a id='item2'></a>

### Exploring Neighborhoods in Toronto

we create a function to explore all the neighborhoods in Toronto

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
# type your answer here


LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude'],
                                  )



Parkwoods
Victoria Village
Harbourfront
Lawrence Heights
Lawrence Manor
Queen's Park
Islington Avenue
Rouge
Malvern
Don Mills North

Woodbine Gardens
Parkview Hill
Ryerson

Garden District

Glencairn

Cloverdale

Islington
Martin Grove

Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South

Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens

Eringate

Markland Wood
Old Burnhamthorpe

Guildwood

Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks

Woburn
Leaside
Central Bay Street

Christie

Cedarbrae

Hillcrest Village
Bathurst Manor
Downsview North

Wilson Heights
Thorncliffe Park
Adelaide

King

Richmond

Dovercourt Village
Dufferin

Scarborough Village
Fairview

Henry Farm
Oriole

Northwood Park
York University
East Toronto
Harbourfront East

Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park

Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East

The Danforth West

Rive

In [27]:
print(toronto_venues.shape)
toronto_venues.head()

(4257, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,Bella Vita Catering & Private Chef Service,43.756651,-79.331524,BBQ Joint
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


Checking how many venues were returned for each neighborhood

In [28]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
Agincourt North,2,2,2,2,2,2
Albion Gardens,8,8,8,8,8,8
Alderwood,9,9,9,9,9,9
Bathurst Manor,18,18,18,18,18,18
Bathurst Quay,17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
Beaumond Heights,8,8,8,8,8,8


In [29]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 264 uniques categories.


<a id='item3'></a>

In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
toronto_onehot.shape

(4257, 264)

#### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [32]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Adelaide,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,Agincourt North,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Albion Gardens,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Alderwood,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,Bathurst Manor,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Bathurst Quay,0.000000,0.0,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
8,Beaumond Heights,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Bedford Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [33]:
toronto_grouped.shape

(205, 264)

In [34]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide
----
         venue  freq
0  Coffee Shop  0.07
1   Restaurant  0.06
2         Café  0.05
3          Bar  0.03
4       Bakery  0.03


----Agincourt----
                       venue  freq
0                     Lounge  0.25
1  Latin American Restaurant  0.25
2             Breakfast Spot  0.25
3             Clothing Store  0.25
4              Metro Station  0.00


----Agincourt North----
               venue  freq
0         Playground   0.5
1               Park   0.5
2        Yoga Studio   0.0
3        Men's Store   0.0
4  Mobile Phone Shop   0.0


----Albion Gardens
----
                  venue  freq
0         Grocery Store  0.25
1            Beer Store  0.12
2   Fried Chicken Joint  0.12
3  Fast Food Restaurant  0.12
4              Pharmacy  0.12


----Alderwood----
            venue  freq
0     Pizza Place  0.22
1             Gym  0.11
2  Sandwich Place  0.11
3        Pharmacy  0.11
4    Skating Rink  0.11


----Bathurst Manor----
           venue  freq
0    Coffee Shop  0.

               venue  freq
0         Food Truck   0.5
1     Baseball Field   0.5
2        Yoga Studio   0.0
3      Metro Station   0.0
4  Mobile Phone Shop   0.0


----Downsview East
----
               venue  freq
0            Airport   0.5
1               Park   0.5
2        Yoga Studio   0.0
3        Men's Store   0.0
4  Mobile Phone Shop   0.0


----Downsview North
----
           venue  freq
0    Coffee Shop  0.11
1           Bank  0.11
2  Shopping Mall  0.06
3    Supermarket  0.06
4     Restaurant  0.06


----Downsview Northwest
----
            venue  freq
0   Grocery Store  0.25
1     Coffee Shop  0.25
2  Discount Store  0.25
3    Liquor Store  0.25
4            Park  0.00


----Downsview West----
           venue  freq
0  Grocery Store  0.33
1           Park  0.17
2          Hotel  0.17
3  Shopping Mall  0.17
4           Bank  0.17


----Dufferin
----
                       venue  freq
0                   Pharmacy  0.12
1                     Bakery  0.12
2              Grocery

                  venue  freq
0            Restaurant  0.25
1   Fried Chicken Joint  0.25
2  Fast Food Restaurant  0.25
3        Sandwich Place  0.25
4         Metro Station  0.00


----Kennedy Park----
              venue  freq
0  Department Store   0.2
1       Bus Station   0.2
2    Discount Store   0.2
3       Coffee Shop   0.2
4        Hobby Shop   0.2


----Kensington Market----
                           venue  freq
0                            Bar  0.07
1          Vietnamese Restaurant  0.06
2                           Café  0.06
3                    Coffee Shop  0.05
4  Vegetarian / Vegan Restaurant  0.05


----King
----
         venue  freq
0  Coffee Shop  0.07
1   Restaurant  0.06
2         Café  0.05
3          Bar  0.03
4       Bakery  0.03


----King and Spadina----
              venue  freq
0   Airport Service  0.18
1  Airport Terminal  0.12
2    Airport Lounge  0.12
3   Harbor / Marina  0.06
4       Coffee Shop  0.06


----King's Mill Park
----
                          

                             venue  freq
0                   Baseball Field   1.0
1                      Yoga Studio   0.0
2  Molecular Gastronomy Restaurant   0.0
3               Mac & Cheese Joint   0.0
4                           Market   0.0


----Oriole
----
                  venue  freq
0        Clothing Store  0.13
1           Coffee Shop  0.08
2  Fast Food Restaurant  0.07
3   Japanese Restaurant  0.05
4         Women's Store  0.03


----Parkdale----
                         venue  freq
0                    Gift Shop  0.15
1                   Restaurant  0.08
2               Breakfast Spot  0.08
3  Eastern European Restaurant  0.08
4                          Bar  0.08


----Parkdale Village----
            venue  freq
0            Café  0.12
1  Breakfast Spot  0.08
2       Nightclub  0.08
3     Coffee Shop  0.08
4     Yoga Studio  0.04


----Parkview Hill----
                venue  freq
0         Pizza Place  0.18
1        Intersection  0.09
2      Breakfast Spot  0.09
3  Athle

                 venue  freq
0                  Pub  0.12
1          Coffee Shop  0.12
2   Light Rail Station  0.06
3  American Restaurant  0.06
4           Restaurant  0.06


----Sunnylea----
                             venue  freq
0                   Baseball Field   1.0
1                      Yoga Studio   0.0
2  Molecular Gastronomy Restaurant   0.0
3               Mac & Cheese Joint   0.0
4                           Market   0.0


----Swansea----
                venue  freq
0         Pizza Place  0.08
1    Sushi Restaurant  0.08
2         Coffee Shop  0.08
3                Café  0.08
4  Italian Restaurant  0.05


----Tam O'Shanter----
                venue  freq
0         Pizza Place  0.14
1         Gas Station  0.07
2            Pharmacy  0.07
3        Noodle House  0.07
4  Chinese Restaurant  0.07


----The Annex----
            venue  freq
0            Café  0.14
1  Sandwich Place  0.14
2     Coffee Shop  0.09
3        Pharmacy  0.05
4    Burger Joint  0.05


----The Beaches--

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Creating a new dataframe and display the top 10 venues for each neighborhood.

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Restaurant,Café,Thai Restaurant,Bakery,Bar,Concert Hall,Vegetarian / Vegan Restaurant,Sushi Restaurant,Office
1,Agincourt,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
2,Agincourt North,Park,Playground,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
3,Albion Gardens,Grocery Store,Beer Store,Fried Chicken Joint,Sandwich Place,Fast Food Restaurant,Pizza Place,Pharmacy,Donut Shop,Doner Restaurant,Dog Run
4,Alderwood,Pizza Place,Pub,Coffee Shop,Gym,Skating Rink,Pharmacy,Athletics & Sports,Sandwich Place,Diner,Deli / Bodega


<a id='item4'></a>

Running *k*-means to cluster the neighborhood into 5 clusters.

In [37]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1])

Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [38]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged=toronto_merged.dropna()

toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].astype('int')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,Parkwoods,43.753259,-79.329656,0,Park,BBQ Joint,Food & Drink Shop,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Ethiopian Restaurant
1,M4A,North York,Victoria Village,Victoria Village,43.725882,-79.315572,1,Portuguese Restaurant,Coffee Shop,Pizza Place,Intersection,Hockey Arena,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,M5A,Downtown Toronto,Harbourfront,Harbourfront,43.654260,-79.360636,1,Coffee Shop,Park,Pub,Mexican Restaurant,Breakfast Spot,Restaurant,Café,Theater,Bakery,Distribution Center
3,M6A,North York,Lawrence Heights,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,1,Furniture / Home Store,Clothing Store,Women's Store,Coffee Shop,Miscellaneous Shop,Boutique,Athletics & Sports,Event Space,Accessories Store,Gift Shop
4,M6A,North York,Lawrence Manor,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,1,Furniture / Home Store,Clothing Store,Women's Store,Coffee Shop,Miscellaneous Shop,Boutique,Athletics & Sports,Event Space,Accessories Store,Gift Shop


### Final Visualisation of clusters

Finally, let's visualize the resulting clusters

<a id='item5'></a>

In [39]:
# create map
map_clusters = folium.Map(location = [latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,-79.329656,0,Park,BBQ Joint,Food & Drink Shop,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Ethiopian Restaurant
37,York,-79.453512,0,Park,Women's Store,Market,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
53,Scarborough,-79.239476,0,Convenience Store,Playground,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
59,East York,-79.338106,0,Park,Convenience Store,Coffee Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
69,North York,-79.464763,0,Park,Airport,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
70,North York,-79.464763,0,Park,Airport,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
88,North York,-79.490074,0,Park,Construction & Landscaping,Bakery,Basketball Court,Dumpling Restaurant,Drugstore,Donut Shop,Eastern European Restaurant,Department Store,Doner Restaurant
89,North York,-79.490074,0,Park,Construction & Landscaping,Bakery,Basketball Court,Dumpling Restaurant,Drugstore,Donut Shop,Eastern European Restaurant,Department Store,Doner Restaurant
90,North York,-79.490074,0,Park,Construction & Landscaping,Bakery,Basketball Court,Dumpling Restaurant,Drugstore,Donut Shop,Eastern European Restaurant,Department Store,Doner Restaurant
111,Central Toronto,-79.388790,0,Park,Bus Line,Swim School,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center,Empanada Restaurant


#### Cluster 2

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,-79.315572,1,Portuguese Restaurant,Coffee Shop,Pizza Place,Intersection,Hockey Arena,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,Downtown Toronto,-79.360636,1,Coffee Shop,Park,Pub,Mexican Restaurant,Breakfast Spot,Restaurant,Café,Theater,Bakery,Distribution Center
3,North York,-79.464763,1,Furniture / Home Store,Clothing Store,Women's Store,Coffee Shop,Miscellaneous Shop,Boutique,Athletics & Sports,Event Space,Accessories Store,Gift Shop
4,North York,-79.464763,1,Furniture / Home Store,Clothing Store,Women's Store,Coffee Shop,Miscellaneous Shop,Boutique,Athletics & Sports,Event Space,Accessories Store,Gift Shop
5,Downtown Toronto,-79.389494,1,Coffee Shop,Yoga Studio,Indian Restaurant,Café,Beer Bar,Discount Store,Bar,Bank,Distribution Center,Italian Restaurant
7,Scarborough,-79.194353,1,Fast Food Restaurant,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
8,Scarborough,-79.194353,1,Fast Food Restaurant,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
9,North York,-79.352188,1,Café,Gym / Fitness Center,Caribbean Restaurant,Japanese Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Dance Studio
10,East York,-79.309937,1,Pizza Place,Fast Food Restaurant,Breakfast Spot,Bank,Gym / Fitness Center,Pharmacy,Gastropub,Athletics & Sports,Intersection,Bus Line
11,East York,-79.309937,1,Pizza Place,Fast Food Restaurant,Breakfast Spot,Bank,Gym / Fitness Center,Pharmacy,Gastropub,Athletics & Sports,Intersection,Bus Line


#### Cluster 3

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Scarborough,-79.160497,2,Bar,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
21,Scarborough,-79.160497,2,Bar,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
22,Scarborough,-79.160497,2,Bar,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


#### Cluster 4

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
97,North York,-79.495697,3,Baseball Field,Food Truck,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Ethiopian Restaurant
105,North York,-79.532242,3,Baseball Field,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
106,North York,-79.532242,3,Baseball Field,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
197,Etobicoke,-79.498509,3,Baseball Field,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
198,Etobicoke,-79.498509,3,Baseball Field,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
199,Etobicoke,-79.498509,3,Baseball Field,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
200,Etobicoke,-79.498509,3,Baseball Field,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
201,Etobicoke,-79.498509,3,Baseball Field,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
202,Etobicoke,-79.498509,3,Baseball Field,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
203,Etobicoke,-79.498509,3,Baseball Field,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


#### Cluster 5

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Etobicoke,-79.554724,4,Home Service,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dance Studio
16,Etobicoke,-79.554724,4,Home Service,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dance Studio
17,Etobicoke,-79.554724,4,Home Service,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dance Studio
18,Etobicoke,-79.554724,4,Home Service,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dance Studio
19,Etobicoke,-79.554724,4,Home Service,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dance Studio
